

R_tibia_15A
Height of Paitent in mm:  1523.5
Length of Paitent AOI (tibia) in mm:  314.8372802734375
AOI Slice Start:  684
AOI Slice End:  893
AOI Slice Range:  210
Mask Slices Normalized to MRI Scans Shape (Purely AOI):  (210, 512, 512)




Preprocessing Data

In [ ]:
from preprocessing import preprocessing

# \\files.auckland.ac.nz\research\resmed202100086-tws ----> Address for raw data
scans_path = 'D:/MRI - Tairawhiti'
# filename_labels = ['R_tibia_15A', 'R_tibia_16A', 'R_tibia_4A']
filename_labels = ['R_tibia_15A']
preprocessed_images, preprocessed_masks = preprocessing(scans_path, filename_labels)

In [9]:
# %pip install custom_layers
%pip install medpy


     -------------------------------------- 151.8/151.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for medpy: filename=MedPy-0.4.0-py3-none-any.whl size=215881 sha256=8af40144571a17ee2b6c3b866921d606562b44942a374826e152907aa5405b89
  Stored in directory: c:\users\ggpc\appdata\local\pip\cache\wheels\41\46\a2\7c585b78f216a3dd8723dbab5f439822fa5dfbff563757a49e
Successfully built medpy


UNet Model

In [ ]:
# ImageNet pretrained DenseNet
from preprocessing import preprocessing

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization, Activation, Conv2DTranspose, concatenate
from keras.layers.core import Dropout
import tensorflow as tf
# from tensorflow.keras.models import Model
from keras.layers import *
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np

# Define U-Net model
def unet_model(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

# Dice Coefficient Loss Function 
def dice_coefficient(y_true, y_pred):
    smooth = 1e-5
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return 1.0 - dice


# Reformat image data structure
training_scans_reshaped = np.concatenate(preprocessed_images, axis=0)
training_scans = training_scans_reshaped.reshape((-1, 512, 512, 1))
train_mask_tibia_labels_reshaped = np.concatenate(preprocessed_masks, axis=0)
train_mask_tibia_labels = train_mask_tibia_labels_reshaped.reshape((-1, 512, 512, 1))

# Split the data into training and validation sets\
images_train, images_val, labels_train, labels_val = train_test_split(training_scans, train_mask_tibia_labels, test_size=0.2, random_state=0)
unseen_scan_model = np.array(training_scans[2][100])
images_train = images_train.astype('float32') / 255.0
images_val = images_val.astype('float32') / 255.0

print(images_train.shape)
print(labels_train.shape)
print(images_train.dtype)
print(labels_train.dtype)
print(images_val.shape)
print(labels_val.shape)
print(images_val.dtype)
print(labels_val.dtype)
print(unseen_scan_model.shape)

# Expand dimensions for the channel (grayscale) dimension
# images_train = np.expand_dims(images_train, axis=-1)
# images_val = np.expand_dims(images_val, axis=-1)
# labels_train = np.expand_dims(labels_train, axis=-1)
# labels_val = np.expand_dims(labels_val, axis=-1)

# Create an instance of the U-Net model
input_shape = (512, 512, 1)  # For grayscale images

# Create an instance of the U-Net model
model = unet_model(input_shape)

# Compile the model
# Binary Cross Entropy Loss Function
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Dice Coefficient Loss Function
# model.compile(optimizer=Adam(), loss=dice_coefficient, metrics=['accuracy'])

# Train the model
# Hyperparameter tuning -> batch_size
model.fit(x=images_train, y=labels_train, batch_size=32, epochs=1, validation_data=(images_val, labels_val))
# Evaluate the model
loss, accuracy = model.evaluate(x=images_val, y=labels_val)

# Perform inference on new, unseen MRI scans
predictions = model.predict(unseen_scan_model)